In [14]:
import numpy as np
import os
import re
import itertools
import collections
import tqdm
import re
from collections import defaultdict
import random
import string

# Treino

In [15]:
def open_arquivo(arquivos):
    corpus=''
    for nome in arquivos:
        with open('./CHAVEFolha/folha95/' + nome, 'r',encoding='iso-8859-1') as f:
            corpus = corpus + f.read()
    return corpus

In [16]:
def regex_extracao(corpus):
    """
    Input: string
    Output: listas de strings"""
    pattern = r"<TEXT>(.*?)</TEXT>"
    corpus = re.sub('\n','',corpus)
    search = re.findall(pattern, corpus, flags=0)
    return search

In [17]:
def pre_processamento(search):
    """
    Input:  listas de strings
    Output: string em lower case e sem pontuações"""
    texto=''
    for i in search:
        i= re.sub('[!"#$%&\'()*+,-:;=?@[\\]^_`{|}~º]','', i)
        i= re.sub('[0123456789]','', i)
        i= re.sub('\x96','', i)
        i= re.sub(r'\s([?.!"](?:\s|$))', r'\1', i)
        i=i.strip()
        i = re.sub('\.(?!$)', ' <s> <s> </s> </s> ', i)
        new = " <s> <s> " + i + " </s> </s> "
        texto=texto+new
    return texto.lower().strip(' ')

In [18]:
def tokenizacao(search):
    """
    Input:  string em lower case e sem pontuações
    Output: lista de palavras"""
    x=[]
    for i in search.split(' '):
        if i!='':
            x.append(i)
    return x

In [19]:
def generate_ngrams(tokens, n):
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [20]:
def unk_selection(token):
    unk_selection=[]
    words=list(set(token))
    for a in range(15):
        unk_selection.append(random.choice(words))
    x=[]
    for i in token:
        if i in list(unk_selection):
            x.append('<unk>')
        else:
            x.append(i)
    print('Palavras substituídas por <unk>:',unk_selection)
    return x

In [21]:
arquivos   = list(os.walk('./CHAVEFolha/folha95/'))[0][2]
arquivos   = arquivos[:100]
corpus     = open_arquivo(arquivos)
search     = regex_extracao(corpus)
texto      = pre_processamento(search)
tokens     = tokenizacao(texto)
tokens     = unk_selection(tokens)
vocabulary = list(set(tokens))

Palavras substituídas por <unk>: ['contrabaixo', 'opcode', 'roupagens', 'pesetas', 'sublimatórias', 'civilpresidente', 'cyclone', 'codex', 'observatory', 'shin', 'paulistério', 'telefonemasno', 'anterioros', 'publicrelations', 'esboçar']


In [22]:
print('Quantidade de palavras diferentes da folha95:',   len(vocabulary))
print('Quantidade de palavras total da folha95     :',     len((tokens)))

Quantidade de palavras diferentes da folha95: 206961
Quantidade de palavras total da folha95     : 4819014


In [23]:
def n_gram(text, n):
    split = text.split()
    grams = [split[i:] for i in range(n)]
    return zip(*grams)

In [24]:
df3 = pd.DataFrame (generate_ngrams(tokens, 3),columns=['3-Gram'])
df2 = pd.DataFrame (generate_ngrams(tokens, 2),columns=['2-Gram'])
df1 = pd.DataFrame (generate_ngrams(tokens, 1),columns=['1-Gram'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
df3.head(15)

,3-Gram
0,<s> <s> datafolha
1,<s> datafolha revela
2,datafolha revela que
3,revela que esperam
4,que esperam governo
5,esperam governo ótimo
6,governo ótimo ou
7,ótimo ou bom
8,ou bom collor
9,bom collor teve


In [26]:
df3 = df3['3-Gram'].value_counts().rename_axis('3-Gram').to_frame('contagem').reset_index()
df2 = df2['2-Gram'].value_counts().rename_axis('2-Gram').to_frame('contagem').reset_index()
df1 = df1['1-Gram'].value_counts().rename_axis('1-Gram').to_frame('contagem').reset_index()

df1_dict = df1.set_index('1-Gram').T.to_dict('list')
df2_dict = df2.set_index('2-Gram').T.to_dict('list')
df3_dict = df3.set_index('3-Gram').T.to_dict('list')

In [27]:
df3.sort_values('contagem', ascending=False).head(15)

,3-Gram,contagem
0,</s> <s> <s>,14301
1,</s> </s> <s>,14301
2,<s> <s> da,2972
3,de são paulo,2926
4,da reportagem local,2681
5,sucursal de brasília,1360
6,da sucursal de,1358
7,<s> da reportagem,1273
8,em são paulo,1260
9,que o governo,1252


In [28]:
df2.sort_values('contagem', ascending=False).head(15)

,2-Gram,contagem
0,</s> </s>,14302
1,<s> <s>,14302
2,</s> <s>,14301
3,que o,11577
4,que a,8098
5,com o,7333
6,para a,7052
7,com a,6981
8,para o,6719
9,são paulo,6372


In [29]:
df1.sort_values('contagem', ascending=False).head(15)

,1-Gram,contagem
0,de,243610
1,a,158724
2,o,150448
3,e,116580
4,que,114307
5,do,97508
6,da,84147
7,em,64439
8,para,53815
9,é,44376


In [30]:
x=[]
for i in df3['3-Gram']:
    x.append(i)
result = map(str.split, x)
df3['3-Gram_list'] = list(tuple(result))

x=[]
for i in range(df3.shape[0]):
    x.append(df3['3-Gram_list'][i][-1])
df3['palavra']=x

x=[]
for i in range(df3.shape[0]):
    a=df3['3-Gram_list'][i][:2]
    x.append(' '.join(a))
df3['contexto'] =x    

x=[]
for i  in (df3['contexto']):
    x.append(df2_dict[i][0])
df3['C(contexto)'] =x

x=[]
for i,j in zip(df3['contagem'],df3['C(contexto)']):
    x.append(i/j)
df3['P(3-Gram)'] = x

df3 = df3[['3-Gram','contagem','palavra','contexto','C(contexto)','P(3-Gram)']]

In [31]:
x=[]
for i in df2['2-Gram']:
    x.append(i)
result = map(str.split, x)
df2['2-Gram_list'] = list(tuple(result))

In [32]:
x=[]
for i in range(df2.shape[0]):
    x.append(df2['2-Gram_list'][i][-1])
df2['palavra']=x

x=[]
for i in range(df2.shape[0]):
    a=df2['2-Gram_list'][i][:1]
    x.append(' '.join(a))
df2['contexto'] =x    

x=[]
for i  in (df2['contexto']):
    x.append(df1_dict[i][0])
df2['C(contexto)'] =x


x=[]
for i,j in zip(df2['contagem'],df2['C(contexto)']):
    x.append(i/j)
df2['P(2-Gram)'] = x

df2 = df2[['2-Gram','contagem','palavra','contexto','C(contexto)','P(2-Gram)']]

In [33]:
qtd_palavras_total = len(tokens)
x=[]
for i in df1['contagem']:
    x.append(i/qtd_palavras_total)
df1['P(1-Gram)'] = x

In [34]:
df3

,3-Gram,contagem,palavra,contexto,C(contexto),P(3-Gram)
0,</s> <s> <s>,14301,<s>,</s> <s>,14301,1.000000
1,</s> </s> <s>,14301,<s>,</s> </s>,14302,0.999930
2,<s> <s> da,2972,da,<s> <s>,14302,0.207803
3,de são paulo,2926,paulo,de são,3741,0.782144
4,da reportagem local,2681,local,da reportagem,2791,0.960588
...,...,...,...,...,...,...
3390789,ter realizado um,1,um,ter realizado,5,0.200000
3390790,novo pontífice se,1,se,novo pontífice,1,1.000000
3390791,bicudo as medidas,1,medidas,bicudo as,1,1.000000
3390792,deixar patente à,1,à,deixar patente,1,1.000000


In [35]:
df2

,2-Gram,contagem,palavra,contexto,C(contexto),P(2-Gram)
0,</s> </s>,14302,</s>,</s>,28604,0.500000
1,<s> <s>,14302,<s>,<s>,28604,0.500000
2,</s> <s>,14301,<s>,</s>,28604,0.499965
3,que o,11577,o,que,114307,0.101280
4,que a,8098,a,que,114307,0.070844
...,...,...,...,...,...,...
1565663,williamsa mclaren,1,mclaren,williamsa,1,1.000000
1565664,necessidade este,1,este,necessidade,678,0.001475
1565665,amaury foi,1,foi,amaury,24,0.041667
1565666,de sedadepois,1,sedadepois,de,243610,0.000004


# Teste

In [37]:
arquivos_teste = list(os.walk('./CHAVEFolha/folha94/'))[0][2]
def open_arquivo_teste(arquivo):
    corpus=''
    with open('./CHAVEFolha/folha94/' + arquivo, 'r',encoding='iso-8859-1') as f:
        corpus = corpus + f.read()
    return corpus

In [38]:
def append_unk(tokens_teste):
    x=[]
    for i in tokens_teste:
        if i not in list(vocabulary):
            x.append('<unk>')
        else:
            x.append(i)
    return x

In [39]:
def contagem_unknown(token_teste):
    contagem_unk=0
    for i in token_teste:
        if i=='<unk>':
            contagem_unk = contagem_unk+1
    return contagem_unk

In [40]:
def calculo_metricas(trigrama, bigrama, unigrama,total):
    prod=0
    i=0
    for trigrama, bigrama, unigrama in zip(trigrama, bigrama, unigrama):
        try:
            p3 = df3_dict[trigrama][0] / df2_dict[bigrama][0] 
        except KeyError:
            p3 = 0.0
        try:
            p2 = df2_dict[bigrama][0] / df1_dict[unigrama][0] 
        except KeyError:
            p2 = 0.0

        p1 = df1_dict[unigrama][0] / total  

        #print((p1,p2,p3))
        prod = prod + np.log2(0.1*p1 + 0.2*p2 + 0.7*p3)
        i=i+1
        #print(prod)
        
    prob_total=(2**prod)
    perplexidade = 1/((prob_total)**(1/i))
    return {'prob_total':prob_total,'perplexidade':perplexidade}

In [44]:
len(arquivos_teste)

365

In [47]:
nome_arquivos        = []
tamanho_vocabulario  = []
qtd_unk_words        = []
probabilidade        = []
perplexidade         = []

total = len((tokens))
for i in arquivos_teste:
    print(i)
    nome_arquivos.append(i)
    
    corpus_teste = open_arquivo_teste(i)
    search_teste = regex_extracao(corpus_teste)
    texto_teste  = pre_processamento(search_teste)
    token_teste  = tokenizacao(texto_teste)
    token_teste  = append_unk(token_teste)
    trigram_teste = generate_ngrams(token_teste, 3)
    bigram_teste  = generate_ngrams(token_teste, 2)
    umgram_teste  = generate_ngrams(token_teste, 1)
    
    tamanho_vocabulario.append(len(set(token_teste)))
    
    qtd_unk_words.append(contagem_unknown(token_teste))
    
    
    metricas = calculo_metricas(trigram_teste, bigram_teste, umgram_teste,total)
    probabilidade.append(metricas['prob_total'])
    perplexidade.append(metricas['perplexidade'])

FSP.940101.sgml


<ipython-input-40-036ff8039f0a>:22: RuntimeWarning: divide by zero encountered in double_scalars
  perplexidade = 1/((prob_total)**(1/i))


FSP.940102.sgml
FSP.940103.sgml
FSP.940104.sgml
FSP.940105.sgml
FSP.940106.sgml
FSP.940107.sgml
FSP.940108.sgml
FSP.940109.sgml
FSP.940110.sgml
FSP.940111.sgml


MemoryError: 

In [52]:
nome_arquivos

['FSP.940101.sgml',
 'FSP.940102.sgml',
 'FSP.940103.sgml',
 'FSP.940104.sgml',
 'FSP.940105.sgml',
 'FSP.940106.sgml',
 'FSP.940107.sgml',
 'FSP.940108.sgml',
 'FSP.940109.sgml',
 'FSP.940110.sgml',
 'FSP.940111.sgml']

In [50]:
tamanho_vocabulario

[8253, 11952, 8279, 8501, 9472, 9399, 9269, 10075, 13337, 9377]

In [51]:
qtd_unk_words

[1187, 2074, 1188, 1331, 1566, 1437, 1346, 2014, 2612, 1507]

In [49]:
probabilidade

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [48]:
perplexidade

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]